<a href="https://colab.research.google.com/github/calarconf/computacion-visual/blob/main/2025-05-14_taller_conversion_formatos_3d/python/Taller8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install trimesh plotly numpy matplotlib


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 711.1/711.1 kB 24.3 MB/s eta 0:00:00


In [ ]:
import trimesh
import numpy as np

def analizar_modelo(mesh, nombre):
    print(f"\n🔍 Análisis de {nombre}:")
    # Información básica
    print(f"- Vértices: {len(mesh.vertices)}")
    print(f"- Caras: {len(mesh.faces)}")

    # Normales (no todos los formatos las incluyen)
    has_normals = mesh.face_normals.any() if hasattr(mesh, 'face_normals') else False
    print(f"- Normales cargadas: {'Sí' if has_normals else 'No'}")

    # Duplicados (vértices no únicos)
    unique_verts = np.unique(mesh.vertices, axis=0)
    print(f"- Vértices duplicados: {len(mesh.vertices) - len(unique_verts)}")

    # Validación de malla
    print(f"- Es cerrado (watertight): {mesh.is_watertight}")
    print(f"- Volumen: {mesh.volume if mesh.is_watertight else 'N/A'}")

# Cargar modelos (¡verifica las rutas!)
mesh_stl = trimesh.load('/content/dode.stl')
mesh_obj = trimesh.load('/content/Barrel_OBJ.obj')
mesh_gltf = trimesh.load('/content/scene.gltf')

# Analizar cada uno
analizar_modelo(mesh_stl, "Modelo STL")
analizar_modelo(mesh_obj, "Modelo OBJ")

# GLTF puede ser una escena (manejar diferente)
if isinstance(mesh_gltf, trimesh.Scene):
    print("\n🔍 Modelo GLTF es una escena con:")
    for i, geom in enumerate(mesh_gltf.geometry.values()):
        analizar_modelo(geom, f"Submalla GLTF-{i}")


🔍 Análisis de Modelo STL:
- Vértices: 640
- Caras: 1440
- Normales cargadas: Sí
- Vértices duplicados: 0
- Es cerrado (watertight): True
- Volumen: 24148.03466365958

🔍 Análisis de Modelo OBJ:
- Vértices: 4572
- Caras: 5664
- Normales cargadas: Sí
- Vértices duplicados: 1376
- Es cerrado (watertight): False
- Volumen: N/A

🔍 Modelo GLTF es una escena con:

🔍 Análisis de Submalla GLTF-0:
- Vértices: 1920
- Caras: 1440
- Normales cargadas: Sí
- Vértices duplicados: 1280
- Es cerrado (watertight): False
- Volumen: N/A


In [ ]:
import plotly.graph_objects as go
import trimesh

def visualizar_con_props(mesh, nombre):
    # Crear figura 3D
    fig = go.Figure(data=[
        go.Mesh3d(
            x=mesh.vertices[:,0],
            y=mesh.vertices[:,1],
            z=mesh.vertices[:,2],
            i=mesh.faces[:,0],
            j=mesh.faces[:,1],
            k=mesh.faces[:,2],
            opacity=0.8,
            color='lightblue'
        )
    ])

    # Añadir propiedades al título
    titulo = f"{nombre}<br>Vértices: {len(mesh.vertices)} | Caras: {len(mesh.faces)}"
    fig.update_layout(title=titulo, scene_aspectmode='data')
    fig.show()

# Visualizar cada modelo
visualizar_con_props(mesh_stl, "STL")
visualizar_con_props(mesh_obj, "OBJ")

# Para GLTF (si es escena):
if isinstance(mesh_gltf, trimesh.Scene):
    for i, geom in enumerate(mesh_gltf.geometry.values()):
        visualizar_con_props(geom, f"GLTF-{i}")

# Carga tus mallas/escena
mesh_stl = trimesh.load('/content/dode.stl')
mesh_obj = trimesh.load('/content/Barrel_OBJ.obj')
mesh_gltf = trimesh.load('/content/scene.gltf')

# Exportar STL → OBJ
mesh_stl.export('dode_converted.obj')

# Exportar OBJ → GLB
# Si mesh_obj es Trimesh:
mesh_obj.export('barrel_converted.glb', file_type='glb')

# Si tienes una escena:
from trimesh.exchange.gltf import export_glb
glb_bytes = export_glb(mesh_gltf)
with open('scene_converted.glb', 'wb') as f:
    f.write(glb_bytes)

# Exportar GLTF → PLY
from trimesh.exchange.ply import export_ply
ply_bytes = export_ply(mesh_gltf, encoding='ascii')
with open('scene_converted.ply', 'wb') as f:
    f.write(ply_bytes)
